# 避障模型训练
通过对上一个示例演示学习，我们掌握了如何收集图片并保存在文件夹中，接下来就需要我们通过神经网络来训练出模型了。
> 首先还是需要我们导入训练所需要的模块：
1. torch:Pytorch是一个方便于初学者学习的深度学习框架，包含有torch和torchvision这两个重要的模块，目前比较流行的深度学习框架有tensorflow，caffe等。
2. torchvision:多用于数据处理和模型处理的模块。

In [ ]:
import torch
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

## 1.数据预处理
现在，我们使用“torchvision”模块里“dataset.ImageFolder()”的方法来创建一个图片数据集。
1. 放入我们之前收集好的图片文件夹“dataset”
2. 通过“torchvision.transforms”对图片数据进行一系列处理，预处理后的数据有助于模型训练。

In [ ]:
dataset = datasets.ImageFolder(
    'datasets',
    transforms.Compose([
        transforms.ColorJitter(0.1, 0.1, 0.1, 0.1),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
)
class_idx = dataset.class_to_idx
print('数据标签与对应的索引',class_idx)

## 2.将数据集分割为训练集和测试集

接下来，我们通过“torch.utils.data.random_split()”方法将数据集拆分为"training"和"test"数据集。
测试集将用于验证我们通过训练集训练的模型的准确性。

In [ ]:
# 划分训练集和测试集
valid_percente = 0.2
num_valid = int(len(dataset)*valid_percente)
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [len(dataset) - num_valid, num_valid])

num_train_dataset = len(train_dataset)
num_test_dataset = len(test_dataset)
print('训练数据集图片数量：',num_train_dataset)
print('测试数据集图片数量：',num_test_dataset)

## 3.创建数据加载器

数据加载器有利于节省计算资源，同时也会提高模型的表现力。
1. 通过“torch.utils.data.DataLoader()”方法来创建数据加载器。
2. 里面主要包括数据集“train_dataset”或者“test_dataset”；
3. 每一次输入神经网络的图片数量:“batch_size”；
4. 是否打乱图片的顺序:“shuffle”；使用芯片的核心数量:“num_workers”。

In [ ]:
batch_size = 16
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=4
)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=4
)


## 4.定义神经网络
“torchvision.models”提供了很多网络结构和预训练模型供我们使用。这里我们使用“alexnet”预训练模型
>“迁移学习”：当“pretrained=True”表示我们会运用“迁移学习”进行训练，我们将一个针对数百万张图像进行训练得到的预训练模型，再结合我们自己的数据进行训练，这样会让我们使用较少的数据就能让模型获得很好的效果，节约很多数据和训练时间。

In [ ]:
model = models.alexnet(pretrained=True)

“alexnet”的预训练模型是通过1000个类别数据集训练得到的，但是我们的数据集只有两个类别！因此我们将模型的分类层（“alexnet”的分类层在第6层网络，不同的网络，分类层的位置也不同。）改为2个类别。

In [ ]:
model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, 2)

最后，我们将模型转移到GPU上执行

In [ ]:
device = torch.device('cuda')
model = model.to(device)

## 5.训练神经网络

接下来就可以通过下面的代码，来训练我们的模型了。

1. 我们将模型训练迭代（epoch）10次，每一次迭代表示模型在整个训练数据集上训练一次，这里表示模型在整个训练数据集上学习了50次；
2. 每训练完一个epoch，就通过我们的测试数据集来对我们的模型进行验证，你可以在训练的时候看到训练的结果；
3. 最后我们把准确率最高的模型保存下来，训练完成之后，我们可以左边的文件管理器看到一个模型文件“ best_model_custom.pth”。

小提示：运行下面的代码会需要一段时间来进行训练，请耐心等待！训练时间会根据你的照片数量，网络结构选择，epoch等因素来决定。

In [ ]:
# 设置训练迭代次数
NUM_EPOCHS = 10
# 设置模型保存路径
model_path = r'students_models/best_model_custom.pth'
# 设置优化器
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
# 开始迭代训练

for epoch in range(NUM_EPOCHS):
    batch = 0
    # 初始化训练参数
    best_accuracy = 0.0
    train_loss = 0.0
    train_corrects = 0

    # 初始化测试参数
    test_acc = 0.0
    test_loss = 0.0
    test_corrects = 0
    # 模型训练
    model.train()
    for images, labels in iter(train_loader):
        # 选择设备将“图片”和“标签”输入模型中
        images = images.to(device)
        labels = labels.to(device)
        # 初始化梯度
        optimizer.zero_grad()
        # 模型前向传播
        outputs = model(images)
        # 通过交叉熵求出模型预测的结果与真实“标签”之间的误差值loss
        tr_loss = F.cross_entropy(outputs, labels)
        # 反向传播，通过loss对模型参数进行求导更新参数
        tr_loss.backward()
        # 使用优化器对模型参数进行更新
        optimizer.step()

        train_loss += tr_loss.item() * images.size(0)

        _, predict = torch.max(outputs, 1)
        train_corrects += torch.sum(labels.data == predict)

    train_loss = train_loss / num_train_dataset
    train_acc = train_corrects.item() / num_train_dataset
    # 对测试集进行评估
    model.eval()
    for images, labels in iter(test_loader):
        images = images.to(device)
        labels = labels.to(device)
        with torch.no_grad():
            # 前向传播得到预测结果
            outputs = model(images)
            _, predict = torch.max(outputs, 1)
            t_loss = F.cross_entropy(outputs, labels)
            test_loss += t_loss.item() * images.size(0)

            # 记录预测失败的数量
            test_corrects += torch.sum(labels.data == predict)

    test_loss = test_loss / num_test_dataset
    test_acc = test_corrects.item() / num_test_dataset

    print('epoch={}'.format(epoch + 1))
    print('训练数据集准确率为：{:.2%}，误差为：{}'.format(train_acc, train_loss))
    print('测试数据集准确率为：{:.2%}, 误差为：{}'.format(test_acc, test_loss))

    if test_acc > best_accuracy:
        torch.save(model.state_dict(), model_path)
        best_accuracy = test_acc
print('训练完成！')